<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/output_parsing/table_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tables QA program

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
%pip install llama-index

In [ ]:
import os

OPENAI_API_TOKEN = "sk-"  # Your OpenAI API token here
os.environ["OPENAI_API_KEY"] = OPENAI_API_TOKEN

In [ ]:
# from llama_index.llms import MockLLM
from llama_index.node_parser import (
    MarkdownElementNodeParser,
)
from llama_index.schema import Document, IndexNode, TextNode


test_table_document = Document(
    text="""
|Model|Size|Code|Commonsense Reasoning|World Knowledge|Reading Comprehension|Math MMLU|BBH|AGI Eval|
|---|---|---|---|---|---|---|---|---|
|MPT|7B|20.5|57.4|41.0|57.5|4.9|26.8|31.0|
|MPT|30B|28.9|64.9|50.0|64.7|9.1|46.9|38.0|
|Falcon|7B|5.6|56.1|42.8|36.0|4.6|26.2|28.0|
|Falcon|40B|15.2|69.2|56.7|65.7|12.6|55.4|37.1|
|Falcon|7B|14.1|60.8|46.2|58.5|6.95|35.1|30.3|
|Llama 1|13B|18.9|66.1|52.6|62.3|10.9|46.9|37.0|
|Llama 1|33B|26.0|70.0|58.4|67.6|21.4|57.8|39.8|
|Llama 1|65B|30.7|70.7|60.5|68.6|30.8|63.4|43.5|
|Llama 1|7B|16.8|63.9|48.9|61.3|14.6|45.3|32.6|
|Llama 2|13B|24.5|66.9|55.4|65.8|28.7|54.8|39.4|
|Llama 2|34B|27.8|69.9|58.7|68.0|24.2|62.6|44.1|
|Llama 2|70B|37.5|71.9|63.6|69.4|35.2|68.9|51.2|

    Table 3: Overall performance on grouped academic benchmarks compared to open-source base models.

    • Popular Aggregated Benchmarks. We report the overall results for MMLU (5 shot) (Hendrycks et al., 2020), Big Bench Hard (BBH) (3 shot) (Suzgun et al., 2022), and AGI Eval (3–5 shot) (Zhong et al., 2023). For AGI Eval, we only evaluate on the English tasks and report the average.

    As shown in Table 3, Llama 2 models outperform Llama 1 models. In particular, Llama 2 70B improves the results on MMLU and BBH by ⇡5 and ⇡8 points, respectively, compared to Llama 1 65B. Llama 2 7B and 30B models outperform MPT models of the corresponding size on all categories besides code benchmarks. For the Falcon models, Llama 2 7B and 34B outperform Falcon 7B and 40B models on all categories of benchmarks. Additionally, Llama 2 70B model outperforms all open-source models.

    In addition to open-source models, we also compare Llama 2 70B results to closed-source models. As shown in Table 4, Llama 2 70B is close to GPT-3.5 (OpenAI, 2023) on MMLU and GSM8K, but there is a significant gap on coding benchmarks. Llama 2 70B results are on par or better than PaLM (540B) (Chowdhery et al., 2022) on almost all benchmarks. There is still a large gap in performance between Llama 2 70B and GPT-4 and PaLM-2-L.

    We also analysed the potential data contamination and share the details in Section A.6.

|Benchmark (shots)|GPT-3.5|GPT-4|PaLM|PaLM-2-L|Llama 2|
|---|---|---|---|---|---|
|MMLU (5-shot)|70.0|86.4|69.3|78.3|68.9|
|TriviaQA (1-shot)|–|–|81.4|86.1|85.0|
|Natural Questions (1-shot)|–|–|29.3|37.5|33.0|
|GSM8K (8-shot)|57.1|92.0|56.5|80.7|56.8|
|HumanEval (0-shot)|48.1|67.0|26.2|–|29.9|
|BIG-Bench Hard (3-shot)|–|–|52.3|65.7|51.2|

    Table 4: Comparison to closed-source models on academic benchmarks. Results for GPT-3.5 and GPT-4 are from OpenAI (2023). Results for the PaLM model are from Chowdhery et al. (2022). Results for the PaLM-2-L are from Anil et al. (2023).

    3 Fine-tuning

    Llama 2-Chat is the result of several months of research and iterative applications of alignment techniques, including both instruction tuning and RLHF, requiring significant computational and annotation resources. In this section, we report on our experiments and findings using supervised fine-tuning (Section 3.1), as well as initial and iterative reward modeling (Section 3.2.2) and RLHF (Section 3.2.3). We also share a new technique, Ghost Attention (GAtt), which we find helps control dialogue flow over multiple turns (Section 3.3). See Section 4.2 for safety evaluations on fine-tuned models.

    
    We initialize our reward models from pretrained chat model checkpoints, as it ensures that both models
    benefit from knowledge acquired in pretraining. In short, the reward model “knows” what the chat model
    ---
    # Statistics of human preference data for reward modeling

| Dataset              | Num. of Comparisons | Avg. # Turns | Avg. # Tokens per Dialogue | Avg. # Tokens per Example | Avg. # Tokens in Prompt | Avg. # Tokens in Response |
|----------------------|----------------------|---------------|-----------------------------|---------------------------|-------------------------|---------------------------|
| Anthropic Helpful    | 122,387              | 3.0           | 251.5                       | 17.7                      | 88.4                    |
| Anthropic Harmless   | 43,966               | 3.0           | 152.5                       | 15.7                      | 46.4                    |
| OpenAI Summarize     | 176,625              | 1.0           | 371.1                       | 336.0                     | 35.1                    |
| OpenAI WebGPT        | 13,333               | 1.0           | 237.2                       | 48.3                      | 188.9                   |
| StackExchange        | 1,038,480            | 1.0           | 440.2                       | 200.1                     | 240.2                   |
| Stanford SHP         | 74,882               | 1.0           | 338.3                       | 199.5                     | 138.8                   |
| Synthetic GPT-J      | 33,139               | 1.0           | 123.3                       | 13.0                      | 110.3                   |
| Meta (Safety & Helpfulness) | 1,418,091     | 3.9           | 798.5                       | 31.4                      | 234.1                   |
| Total                | 2,919,326            | 1.6           | 595.7                       | 108.2                     | 216.9                   |

    Table 6: Statistics of human preference data for reward modeling. We list both the open-source and internally collected human preference data used for reward modeling. Note that a binary human preference comparison contains 2 responses (chosen and rejected) sharing the same prompt (and previous dialogue). Each example consists of a prompt (including previous dialogue if available) and a response, which is the input of the reward model. We report the number of comparisons, the average number of turns per dialogue, the average number of tokens per example, per prompt and per response. More details on Meta helpfulness and safety data per batch can be found in Appendix A.3.1.

    Training Objectives  
    To train the reward model, we convert our collected pairwise human preference data into a binary ranking label format (i.e., chosen & rejected) and enforce the chosen response to have a higher score than its counterpart. We used a binary ranking loss consistent with Ouyang et al. (2022):

        """
)

## Baseline 1: Using MarkDown Raw Output (containing multipel tables) as Input for Query Engine

In [ ]:
from llama_index import (
    VectorStoreIndex,
    get_response_synthesizer,
)
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine

# build index
index = VectorStoreIndex.from_documents([test_table_document])

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
)
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
)

# query different questions
response_1 = query_engine.query(
    "What is MPT 30b performance for common sense reasoning?"
)
print(response_1)

response_2 = query_engine.query("What is PaLM-2-L performance for TriviaQA?")
print(response_2)


response_3 = query_engine.query("What is LLAMA 2 performance for HumanEval?")
print(response_3)

response_4 = query_engine.query("What is LLAMA 2 performance for AGI Eval?")
print(response_4)

response_5 = query_engine.query(
    "What is LLAMA 2 performance for HumanEval and AGI Eval?"
)
print(response_5)

The performance of MPT 30B for common sense reasoning is 64.9.
PaLM-2-L performance for TriviaQA is 86.1.
Llama 2's performance for HumanEval is 29.9.
Llama 2's performance for AGI Eval is not mentioned in the given context.
LLAMA 2's performance for HumanEval and AGI Eval is not mentioned in the given context.


## Baseline 2: Apply `MarkdownElementNodeParser` for parsing table nodes

In [ ]:
node_parser = MarkdownElementNodeParser()

nodes = node_parser.get_nodes_from_documents([test_table_document])

Embeddings have been explicitly disabled. Using MockEmbedding.


0it [00:00, ?it/s]

2it [00:15,  7.67s/it]


In [ ]:
print(len(nodes))

6


In [ ]:
base_nodes, node_mappings = node_parser.get_base_nodes_and_mappings(nodes)

# for node in node_mappings:
#     print("[Node maping]-----------------")

# print(node)
# print(node_mappings[node])
# print(node.metadata)

for node in base_nodes:
    print("[Base Node]---")
    # for d in node.metadata:
    #     print(d)
    # print(node.metadata)
    print(node.text)

# for node in nodes:
#     print("[node]---")
#     print(node)

[Base Node]---
This table compares different models based on various metrics such as commonsense reasoning, world knowledge, reading comprehension, math MMLU, BBH, and AGI evaluation. It also compares benchmark performance of GPT-3.5, GPT-4, PaLM, PaLM-2-L, and Llama 2 on different tasks such as MMLU, TriviaQA, Natural Questions, GSM8K, HumanEval, and BIG-Bench Hard.,
with the following table title:
Comparison of Models and Benchmark Performance,
with the following columns:
- Model: The name of the model
- Size: The size of the model
- Code: The code metric
- Commonsense Reasoning: The commonsense reasoning metric
- World Knowledge: The world knowledge metric
- Reading Comprehension: The reading comprehension metric
- Math MMLU: The math MMLU metric
- BBH: The BBH metric
- AGI Eval: The AGI evaluation metric

[Base Node]---
Table 3: Overall performance on grouped academic benchmarks compared to open-source base models.

    • Popular Aggregated Benchmarks. We report the overall results

## Baseline 2: Table Retrieval and QA using recursive retrieval and query engine

In [ ]:
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index import VectorStoreIndex
from llama_index.embeddings import TogetherEmbedding, OpenAIEmbedding
from llama_index.llms import OpenAI
from llama_index.service_context import ServiceContext

# construct top-level vector index + query engine
vector_index = VectorStoreIndex(nodes)
vector_retriever = vector_index.as_retriever(similarity_top_k=3)
vector_query_engine = vector_index.as_query_engine(similarity_top_k=3)

from llama_index.retrievers import RecursiveRetriever

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    node_dict=node_mappings,
    verbose=True,
)

llm = OpenAI(temperature=0, model="gpt-4")
service_context = ServiceContext.from_defaults(llm=llm)

recursive_query_engine = RetrieverQueryEngine.from_args(
    recursive_retriever, service_context=service_context
)

In [ ]:
# query different questions
response_1 = recursive_query_engine.query(
    "What is MPT 30b performance for common sense reasoning?"
)
print(response_1)

response_2 = recursive_query_engine.query(
    "What is PaLM-2-L performance for TriviaQA?"
)
print(response_2)


response_3 = recursive_query_engine.query(
    "What is LLAMA 2 performance for HumanEval?"
)
print(response_3)

response_4 = recursive_query_engine.query(
    "What is LLAMA 2 performance for AGI Eval?"
)
print(response_4)

response_5 = recursive_query_engine.query(
    "What is LLAMA 2 performance for HumanEval and AGI Eval?"
)
print(response_5)

Retrieving with query id None: What is MPT 30b performance for common sense reasoning?
Retrieving text node: Table 20: Performance on standard benchmarks.
Retrieving text node: This table provides information on different models and their performance in various benchmarks.,
with the following table title:
Model Performance in Benchmarks,
with the following columns:
- Model: The name of the model
- Size: The size of the model
- Code: The code performance of the model
- Commonsense Reasoning: The performance of the model in commonsense reasoning tasks
- World Knowledge: The performance of the model in world knowledge tasks
- Reading Comprehension: The performance of the model in reading comprehension tasks
- Math MMLU: The performance of the model in math MMLU tasks
- BBH: The performance of the model in BBH tasks
- AGI Eval: The performance of the model in AGI evaluation

|Benchmark (shots)|GPT-3.5|GPT-4|PaLM|PaLM-2-L|Llama 2|
|---|---|---|---|---|---|
|MMLU (5-shot)|70.0|86.4|69.3|78.3

In [ ]:
print(response_1)
print(response_2)
print(response_3)
print(response_4)
print(response_5)

The performance of the MPT 30B model for commonsense reasoning is 64.9.
The context does not provide information on the performance of the PaLM-2-L model for TriviaQA.
The context does not provide specific information on Llama 2's performance on the HumanEval benchmark.
The performance of the Llama 2 model for AGI Eval varies depending on the size of the model. For the 13B model, the score is 39.4. For the 34B model, the score is 44.1. The 70B model has the highest score at 51.2.
The exact performance metrics for Llama 2 on HumanEval and AGI Eval are not provided in the context. However, it is mentioned that Llama 2 models generally outperform Llama 1 models and other open-source models on various benchmarks. For AGI Eval, only the English tasks were evaluated and the average results were reported. On the other hand, there is a significant gap in performance between Llama 2 and some closed-source models on coding benchmarks, which might include HumanEval.


## Baseline 3 Text-to-pandas query engine for Table Nodes

In [ ]:
import json
import pandas as pd
import ast

table_dfs = []
for node in nodes:
    if "table" in node.id_:
        if "table_df" in node.metadata:
            table_dfs.append(
                pd.DataFrame.from_dict(
                    ast.literal_eval(node.metadata["table_df"])
                )
            )
            # table_dfs.append(node.metadata["table_df"])
print(len(table_dfs))
table_dfs[0]

2


,Model,Size,Code,Commonsense Reasoning,World Knowledge,Reading Comprehension,Math MMLU,BBH,AGI Eval
0,MPT,7B,20.5,57.4,41.0,57.5,4.90,26.8,31.0
1,MPT,30B,28.9,64.9,50.0,64.7,9.10,46.9,38.0
2,Falcon,7B,5.6,56.1,42.8,36.0,4.60,26.2,28.0
3,Falcon,40B,15.2,69.2,56.7,65.7,12.60,55.4,37.1
4,Falcon,7B,14.1,60.8,46.2,58.5,6.95,35.1,30.3
5,Llama 1,13B,18.9,66.1,52.6,62.3,10.90,46.9,37.0
6,Llama 1,33B,26.0,70.0,58.4,67.6,21.40,57.8,39.8
7,Llama 1,65B,30.7,70.7,60.5,68.6,30.80,63.4,43.5
8,Llama 1,7B,16.8,63.9,48.9,61.3,14.60,45.3,32.6
9,Llama 2,13B,24.5,66.9,55.4,65.8,28.70,54.8,39.4


## Create Pandas Query Engines

We create a pandas query engine over each structured table.

These can be executed on their own to answer queries about each table.

In [ ]:
import logging
import sys
from IPython.display import Markdown, display

import pandas as pd
from llama_index.query_engine import PandasQueryEngine
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.query_engine import PandasQueryEngine, RetrieverQueryEngine
from llama_index.retrievers import RecursiveRetriever
from llama_index.schema import IndexNode
from llama_index.llms import OpenAI

# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
# define query engines over these tables
llm = OpenAI(model="gpt-4")

service_context = ServiceContext.from_defaults(llm=llm)
df_query_engines = [
    PandasQueryEngine(table_df, service_context=service_context)
    for table_df in table_dfs
]
response_1 = df_query_engines[0].query(
    "What is MPT 30b performance for common sense reasoning?"
)
print(response_1)

64.9
